**대규모 텐서플로 모델 훈련과 배포**

# 설정

In [1]:
import tensorflow as tf

tf.config.list_physical_devices('GPU')

2025-07-27 15:05:17.661103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753628717.801416      32 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753628717.842957      32 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753628718.164848      32 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753628718.164906      32 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753628718.164909      32 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 텐서플로 모델 서빙하기

먼저 TF 서빙을 사용하여 모델을 배포한 다음 Google Vertex AI에 배포해 본다.

## 텐서플로 서빙 사용

가장 먼저 모델을 빌드하고 학습한 다음 SavedModel 포맷으로 내보낸다.

### SavedModel 내보내기

MNIST 데이터 세트를 로드하고, 스케일을 조정하고, 분할한다.

In [30]:
from pathlib import Path

mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(1 / 255),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(optimizer="SGD", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
model.export(model_path)

Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7268 - loss: 1.0815 - val_accuracy: 0.9016 - val_loss: 0.3679
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8977 - loss: 0.3685 - val_accuracy: 0.9170 - val_loss: 0.2969
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9150 - loss: 0.3053 - val_accuracy: 0.9288 - val_loss: 0.2661
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9229 - loss: 0.2786 - val_accuracy: 0.9346 - val_loss: 0.2425
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9301 - loss: 0.2447 - val_accuracy: 0.9378 - val_loss: 0.2246
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9355 - loss: 0.2282 - val_accuracy: 0.9406 - val_loss: 0.2107
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9386 - loss: 0.2225 - val_accuracy: 0.9440 - val_loss: 0.1971
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9430 - loss: 0.2034 - 

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=my_mnist_model/0001.

파일 트리를 살핀다:

In [32]:
sorted([str(path) for path in model_path.parent.glob("**/*")])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

SavedModel을 검사한다:

In [33]:
!saved_model_cli show --dir '{model_path}'

2025-07-27 16:06:38.258039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753632398.275238    2216 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753632398.280470    2216 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753632398.295325    2216 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753632398.295361    2216 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753632398.295367    2216 computation_placer.cc:177] computation placer alr